In [ ]:
from elasticsearch import Elasticsearch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import re
import math
from tqdm import tqdm
import numpy as np

import torch

In [2]:
df = pd.read_json('../../data/wikipedia_documents.json').T

In [3]:
df.head()

,text,corpus_source,url,domain,title,author,html,document_id
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,TODO,None,나라 목록,None,None,0
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,TODO,None,나라 목록,None,None,1
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,TODO,None,백남준,None,None,2
3,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",위키피디아,TODO,None,아오조라 문고,None,None,3
4,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,위키피디아,TODO,None,아오조라 문고,None,None,4


In [4]:
df = df.drop_duplicates(subset=['text'])
df.nunique()

text             56737
corpus_source        1
url                  1
domain               0
title            31726
author               0
html                 0
document_id      56737
dtype: int64

In [5]:
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

In [6]:
df['text'][60]

'KETEL의 초기화면\n\n\n\u3000CONNECT 240\n\u3000[[[[[[[\n\u3000### ketel에 연결되었습니다.(korea:pc\n\u3000### Welcome to ketel!!\n\u3000### 서비스 명에 ketel을 입력하십시오\n\u3000### Please type "ke" for KETEL English Service\n\u3000서비스 명(login):kete\n\n\n\u3000\u3000\u3000\u3000\u3000\u3000\u3000KK\u3000 KKK\u3000\u3000EEEEE\u3000TTTTTTTT\u3000\u3000EEEEEE\u3000 Lt\n\u3000\u3000\u3000\u3000\u3000\u3000 KK\u3000KK\u3000\u3000\u3000EE\u3000\u3000\u3000\u3000 \u3000TT\u3000\u3000\u3000 EE\u3000\u3000\u3000\u3000\n\u3000\u3000\u3000\u3000\u3000\u3000KKK\u3000\u3000\u3000\u3000 EEEE\u3000\u3000\u3000\u3000TT\u3000\u3000\u3000EEEE\u3000\u3000 L\n\u3000\u3000\u3000\u3000\u3000 KK KK\u3000\u3000\u3000 EE\u3000\u3000\u3000\u3000\u3000TT\u3000\u3000\u3000EE\u3000\u3000\u3000\u3000L\n\u3000\u3000\u3000\u3000\u3000KK\u3000 KKK\u3000EEEEEE\u3000\u3000 TT\u3000\u3000\u3000EEEEEE\u3000 LLLLLLL\n\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000 <<<알립니다>>\n\u30001.<<KETEL 정보서비스 일시 정지 안내>\n\u30002.KETE

In [7]:
len(tokenizer.tokenize(df['text'][60]))

440

In [8]:
end_of_sentences = re.findall(r'[가-힣]+\S*\.', df['text'][60])

In [9]:
sentences = re.split(r'[가-힣]+\S\.', df['text'][60])

In [10]:
print(end_of_sentences, len(end_of_sentences))
print(sentences, len(sentences))

['연결되었습니다.', '마감...'] 2
['KETEL의 초기화면\n\n\n\u3000CONNECT 240\n\u3000[[[[[[[\n\u3000### ketel에 ', '(korea:pc\n\u3000### Welcome to ketel!!\n\u3000### 서비스 명에 ketel을 입력하십시오\n\u3000### Please type "ke" for KETEL English Service\n\u3000서비스 명(login):kete\n\n\n\u3000\u3000\u3000\u3000\u3000\u3000\u3000KK\u3000 KKK\u3000\u3000EEEEE\u3000TTTTTTTT\u3000\u3000EEEEEE\u3000 Lt\n\u3000\u3000\u3000\u3000\u3000\u3000 KK\u3000KK\u3000\u3000\u3000EE\u3000\u3000\u3000\u3000 \u3000TT\u3000\u3000\u3000 EE\u3000\u3000\u3000\u3000\n\u3000\u3000\u3000\u3000\u3000\u3000KKK\u3000\u3000\u3000\u3000 EEEE\u3000\u3000\u3000\u3000TT\u3000\u3000\u3000EEEE\u3000\u3000 L\n\u3000\u3000\u3000\u3000\u3000 KK KK\u3000\u3000\u3000 EE\u3000\u3000\u3000\u3000\u3000TT\u3000\u3000\u3000EE\u3000\u3000\u3000\u3000L\n\u3000\u3000\u3000\u3000\u3000KK\u3000 KKK\u3000EEEEEE\u3000\u3000 TT\u3000\u3000\u3000EEEEEE\u3000 LLLLLLL\n\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000 <<<알립니다>>\n\u30001.<<KETEL 정보서비스 일시

In [11]:
max_possible_length = 510
partial_text = ''
for i in range(len(sentences)-1):
  
  # print(len(tokenizer.tokenize(sentences[i] + end_of_sentences[i])))
  max_possible_length -= len(tokenizer.tokenize(sentences[i] + end_of_sentences[i]))
  if max_possible_length >= 0:
    partial_text += sentences[i] + end_of_sentences[i]
  else:
    

SyntaxError: unexpected EOF while parsing (2632368297.py, line 10)

In [71]:
splited_documents = []

In [72]:
count = 0
for index, document in tqdm(df.iterrows()):
  # print("==================count=====================",count)
  
  original_passage = document['text']

  length_of_tokenized_text = len(tokenizer.tokenize(original_passage))

  number_of_passages_to_create = math.ceil(length_of_tokenized_text / 400)  # 510으로 안하는 이유는 110만큼의 buffer를 주기 위해서
  min_required_length = length_of_tokenized_text // number_of_passages_to_create  # 최소 길이
  max_possible_length = min_required_length + 110  # 최대길이

  end_of_sentences = re.findall(r'[가-힣]+\S*\.', original_passage)
  sentences = re.split(r'[가-힣]+\S*\.', original_passage)
  sentence_counter = 0


  # print(min_required_length)
  for i in range(number_of_passages_to_create):
    partial_text = ''
    partial_text_length = 0

    if sentence_counter == len(end_of_sentences)-1:
      break
    
    for j in range(len(sentences) - 1):
      partial_text += sentences[sentence_counter] + end_of_sentences[sentence_counter]
      partial_text_length += len(tokenizer.tokenize(sentences[sentence_counter] + end_of_sentences[sentence_counter]))
      # print('==============================', sentence_counter, len(sentences)-1, len(end_of_sentences)-1)
      # print(partial_text, partial_text_length)
      sentence_counter += 1
      if partial_text_length >= min_required_length or sentence_counter == len(end_of_sentences)-1:
        
        if sentence_counter == len(end_of_sentences)-1:
          partial_text += sentences[-1]
        document_to_be_added = {
          'text': partial_text,
          'title': document['title'],
          'document_id': count # document['document_id']
        }

        splited_documents.append(document_to_be_added)
        count +=1
        break
  



  #   if length_of_tokenized_text < 510:
  #     document_to_be_added = {
  #       'text': document['text'],
  #       'title': document['title'],
  #       'document_id': document['document_id']
  #     }

  #     splited_documents.append(document_to_be_added)
    
  #   else:

  
  # count += 1
  # if count > 5:
  #   break

56737it [03:08, 301.69it/s]


In [14]:
len(splited_documents)

78411

In [7]:
splited_documents[7504]

NameError: name 'splited_documents' is not defined

In [37]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es.ping()

True

In [141]:
es.indices.delete(index='wiki_documents_dense', ignore=[400, 404])

{'acknowledged': True}

In [142]:
res = es.indices.get_alias("*")
for Name in res:
  print(Name)

.kibana-event-log-7.15.1-000001
.apm-custom-link
wiki_documents_splited_dense
.apm-agent-configuration
wiki_documents
.kibana_7.15.1_001
.tasks
.kibana_task_manager_7.15.1_001


/tmp/ipykernel_11647/3052314421.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.indices.get_alias("*")
/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: this request accesses system indices: [.apm-custom-link, .apm-agent-configuration, .kibana_7.15.1_001, .tasks, .kibana_task_manager_7.15.1_001], but in a future major version, direct access to system indices will be prevented by default
  warnings.warn(message, category=ElasticsearchWarning)


In [143]:
def generator(splited_documents):
  for c, line in enumerate(splited_documents):
    yield {
      '_index': 'wiki_documents_splited',
      '_type': '_doc',
      '_id': line.get('document_id', None),
      '_source': {
        'title': line.get('title', ''),
        'text': line.get('text', '')
      }
    }
  raise StopIteration

In [144]:
CONFIG = {
  'settings':{
    'number_of_shards':1,
    'number_of_replicas':1,
    'analysis':{
      'analyzer': {
        'nori_analyzer':{
          'type': 'custom',
          'tokenizer': 'nori_tokenizer'
        }
      },
    }
  },
  'mappings':{
    'properties':{
      'title':{
        'type':'text'
      },
      'text':{
        'type':'text',
        "analyzer": "nori_analyzer"
      }
    }
  }
}

In [145]:
es.indices.create(index='wiki_documents_splited', ignore=[400,404], body=CONFIG)

/tmp/ipykernel_11647/923970138.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index='wiki_documents_splited', ignore=[400,404], body=CONFIG)


{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'wiki_documents_splited'}

In [146]:
from elasticsearch import helpers
gen = generator(splited_documents)

In [147]:
try:
  res = helpers.bulk(es, gen, chunk_size = 1)
  # print(res)
except Exception as e:
  print(e)
  print('Done')

generator raised StopIteration
Done


In [84]:
p_encoder = AutoModel.from_pretrained('encoders/p_encoder').to('cuda')

In [82]:
es.count(index = 'wiki_documents_splited')

{'count': 78410,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [88]:
splited_documents[0]

{'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다.',
 'title': '나라 목록',
 'document_id': 0}

In [128]:
p_embs = []
for document in tqdm(splited_documents):
  with torch.no_grad():
    p_encoder.eval()
    p_val = tokenizer([document['text']], padding="max_length", truncation=True, return_tensors='pt').to('cuda')

    p_emb = p_encoder(**p_val)
    p_embs.append(p_emb[1].cpu().detach().numpy().tolist()[0])

100%|██████████| 78411/78411 [20:32<00:00, 63.60it/s]


In [ ]:
p_emb[1].cpu().detach().numpy().tolist()[0]

In [117]:
len(vec)

768

In [133]:
CONFIG = {
  'settings':{
    'number_of_shards':1,
    'number_of_replicas':1,
    'analysis':{
      'analyzer': {
        'nori_analyzer':{
          'type': 'custom',
          'tokenizer': 'nori_tokenizer'
        }
      },
    }
  },
  'mappings':{
    'properties':{
      'text':{
        'type':'text',
        "analyzer": "nori_analyzer"
      },
      'vector':{
        'type':'dense_vector',
        'dims':768
      }
    }
  }
}

In [134]:
es.indices.create(index='wiki_documents_splited_dense', ignore=[400,404], body=CONFIG)

/tmp/ipykernel_11647/2560678228.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index='wiki_documents_splited_dense', ignore=[400,404], body=CONFIG)


{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'wiki_documents_splited_dense'}

In [135]:
def generator(splited_documents, p_embs):
  for document, vector in zip(splited_documents, p_embs):
    yield {
      '_index': 'wiki_documents_splited_dense',
      '_type': '_doc',
      '_id': document.get('document_id', None),
      '_source': {
        'text': document.get('text', ''),
        'vector': vector
      }
    }
  raise StopIteration

In [136]:
gen = generator(splited_documents, p_embs)
try:
  res = helpers.bulk(es, gen, chunk_size = 1)
  # print(res)
except Exception as e:
  print(e)
  print('Done')

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


generator raised StopIteration
Done


In [149]:
es.get(index='wiki_documents_splited_dense', id=0)

{'_index': 'wiki_documents_splited_dense',
 '_type': '_doc',
 '_id': '0',
 '_version': 1,
 '_seq_no': 0,
 '_primary_term': 1,
 'found': True,
 '_source': {'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다.',
  'vector': [0.9344813227653503,
   -0.3079654574394226,
   0.9136772751808167,
   -0.999625027179718,
   0.7775430083274841,
   0.602023720741272,
   -0.9999997615814209,
   -0.13547107577323914,
   0.8865708708763123,
   0.16889478266239166,
   -0.4940008223056793,
   0.9369648098945618,
   -0.6527087688446045,
   -0.5305598974227905,
   0.7570145726203918,
   -0.9999809861183167,
   -0.9509158134460

In [187]:
q_encoder = AutoModel.from_pretrained('encoders/q_encoder').to('cuda')

In [194]:
with torch.no_grad():
  q_encoder.eval()
  q_val = tokenizer(['미국의 대통령은 누굴까?'], padding="max_length", truncation=True, return_tensors='pt').to('cuda')

  q_emb = q_encoder(**q_val)
  q_output = q_emb[1].cpu().detach().numpy().tolist()[0]

In [195]:
q_output

[0.6697671413421631,
 -0.6526841521263123,
 0.4279637932777405,
 -0.9571647644042969,
 -0.6496824622154236,
 0.2592073976993561,
 -0.7596632838249207,
 0.2635655403137207,
 0.633450984954834,
 -0.6572563648223877,
 -0.19874678552150726,
 -0.8489906787872314,
 0.22504743933677673,
 0.28977158665657043,
 0.2709265351295471,
 -0.4014929234981537,
 -0.9442694783210754,
 0.09671510010957718,
 0.9969265460968018,
 0.2728358209133148,
 0.20214395225048065,
 -0.6018244028091431,
 -0.40401744842529297,
 0.9840426445007324,
 -0.16704581677913666,
 0.4025427997112274,
 -0.6709930896759033,
 -0.9219578504562378,
 0.08662321418523788,
 -0.30048367381095886,
 0.4342647194862366,
 -0.9226238131523132,
 0.5183486342430115,
 0.9900227785110474,
 0.19196340441703796,
 0.13610520958900452,
 -0.9502851366996765,
 -0.9788157343864441,
 0.2615227997303009,
 0.31027594208717346,
 0.3470304310321808,
 -0.8298472762107849,
 -0.9935808181762695,
 0.9907791018486023,
 0.9882649183273315,
 0.14615939557552338,
 0

In [196]:
es.search(
  index='wiki_documents_splited_dense',
  body={
    'query':{
      "script_score": {
        "query" : {
          "match_all" : {}
        },
        "script": {
          # "source": "1 / (1 + l2norm(params.queryVector, doc['vector']))",
          "source": "cosineSimilarity(params.queryVector, doc['vector']) + 1.0",
          "params": {
            "queryVector": q_output
          }
        }
      }
    }
  },
  size= 3
)

/tmp/ipykernel_11647/3471425228.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.search(


{'took': 178,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 1.636196,
  'hits': [{'_index': 'wiki_documents_splited_dense',
    '_type': '_doc',
    '_id': '54810',
    '_score': 1.636196,
    '_source': {'text': '파일:JoeClark.jpg|미국 미국<br />지미 카터, <br />대통령\n파일:Roy Jenkins 1977.jpg|EU 유럽 연합<br />로이 젱킨스, 의장',
     'vector': [0.6940145492553711,
      -0.46798351407051086,
      0.8579187989234924,
      -0.9955738186836243,
      0.4176998734474182,
      0.15787911415100098,
      -0.9989945888519287,
      -0.22609156370162964,
      0.9678066968917847,
      -0.4837448000907898,
      -0.34187498688697815,
      0.20611472427845,
      -0.6092385649681091,
      -0.3599953353404999,
      0.7680521607398987,
      -0.993483304977417,
      -0.9781216979026794,
      -0.6486918926239014,
      0.9554654955863953,
      0.8859413862228394,
      0.42767104506492615,
     